In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

import torch


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!cp -r drive/MyDrive/diploma/deepset/* .

In [ ]:
!unzip processed_data.zip   

In [5]:
class CONFIG:
    # DATA
    features = 16
    instances = 64
    classes = 2

    # TRAIN
    num_epochs = 50
    train_batch_size = 1024
    learning_rate = 0.002
    criterion = 'torch.nn.BCEWithLogitsLoss'
    FIRST_DROPOUT_RATE = 0.7
    SECOND_DROPOUT_RATE = 0.5

    test_batch_size = 1024

    # MODEL
    hidden_size_0=64
    hidden_size_1=128
    predlast_hidden_size=256
    meta_size=27
    out_classes=3
    standardscaler=True
    meta_standardscaler=True
    transpose=True

    # device = torch.device("cuda:0" if torch.cuda.is_available() else if torch.backends.mps.is_available() then "mps" else "cpu")
    # device is cuda is available else mps is available else cpu
    device_mps_or_cpu = ("mps" if torch.backends.mps.is_available() else "cpu")
    device = torch.device("cuda:0" if torch.cuda.is_available() else device_mps_or_cpu)

    # OTHER
    seed = 42
    num_workers = 8


## Model

### model v1

In [ ]:
from deepsets import deepsetlayers
import torch.nn.functional as F

class DeepSetModel(torch.nn.Module):
    def __init__(self, hidden_size_0=1, hidden_size_1=1, predlast_hidden_size=1, meta_size = 27, out_classes=1):
        super().__init__()
        self.hidden_size_0 = hidden_size_0
        self.hidden_size_1 = hidden_size_1
        self.predlast_hidden_size = predlast_hidden_size
        self.meta_size = meta_size
        self.out_classes = out_classes
        self.inv_0 = deepsetlayers.InvLinear(1, hidden_size_0)
        self.inv_1 = deepsetlayers.InvLinear(hidden_size_0, hidden_size_1)
        self.regressor = torch.nn.Sequential(
            torch.nn.Linear(hidden_size_1 + meta_size, predlast_hidden_size),
            torch.nn.ReLU(),
            torch.nn.Linear(predlast_hidden_size, out_classes),
        )

    def forward(self, x, y):
        x = x.unsqueeze(-1)
        # (batch_size, N, M, 1)
        N = x.shape[1]
        x = x.flatten(0, 1)
        # (batch_size * N, M, 1)
        x = self.inv_0(x)
        # (batch_size * N, hidden_size_0)
        x = x.reshape(-1, N, self.hidden_size_0)
        # (batch_size, N, hidden_size_0)
        x = self.inv_1(x)
        # (batch_size, hidden_size_1)

        # (batch_size, meta_size)
        y = y.view(-1, self.meta_size)
        

        x = torch.hstack([x, y])
        # (batch_size, hidden_size_1 + meta_size)
        x = self.regressor(x)
        # (batch_size, out_classes)
        return x

### model v2

In [ ]:
from deepsets import deepsetlayers
import torch.nn.functional as F

class DeepSetModelV2(torch.nn.Module):
    def __init__(self, hidden_size_0=1, hidden_size_1=1, predlast_hidden_size=1, meta_size = 27, out_classes=1):
        super().__init__()
        self.hidden_size_0 = hidden_size_0
        self.hidden_size_1 = hidden_size_1
        self.predlast_hidden_size = predlast_hidden_size
        self.meta_size = meta_size
        self.out_classes = out_classes
        self.inv_0 = deepsetlayers.InvLinear(hidden_size_0, hidden_size_0)
        self.inv_1 = deepsetlayers.InvLinear(hidden_size_1, hidden_size_1)
        self.equiv_0 = deepsetlayers.EquivLinear(1, hidden_size_0)
        self.equiv_1 = deepsetlayers.EquivLinear(hidden_size_0, hidden_size_1)
        self.relu = torch.nn.ReLU()
        self.regressor = torch.nn.Sequential(
            torch.nn.Linear(hidden_size_1 + meta_size, predlast_hidden_size),
            torch.nn.ReLU(),
            torch.nn.Linear(predlast_hidden_size, out_classes),
        )

    def forward(self, x, y):
        x = x.unsqueeze(-1)
        # (batch_size, N, M, 1)
        N = x.shape[1]
        x = x.flatten(0, 1)
        # (batch_size * N, M, 1)
        x = self.equiv_0(x)
        # (batch_size * N, M, hidden_size_0)
        x = self.relu(x)
        x = self.inv_0(x)
        # (batch_size * N, hidden_size_0)
        x = x.reshape(-1, N, self.hidden_size_0)
        # (batch_size, N, hidden_size_0)

        x = self.equiv_1(x)
        # (batch_size, N, hidden_size_1)
        x = self.relu(x)
        x = self.inv_1(x)
        # (batch_size, hidden_size_1)

        y = y.view(-1, self.meta_size)
        # (batch_size, meta_size)        

        x = torch.hstack([x, y])
        # (batch_size, hidden_size_1 + meta_size)
        x = self.regressor(x)
        # (batch_size, out_classes)
        return x

### modelv3

In [ ]:
from deepsets import deepsetlayers
import torch.nn.functional as F

class DeepSetModelV3(torch.nn.Module):
    def __init__(self, hidden_size_0=1, hidden_size_1=1, predlast_hidden_size=1, meta_size = 27, out_classes=1):
        super().__init__()
        self.hidden_size_0 = hidden_size_0
        self.hidden_size_1 = hidden_size_1
        self.predlast_hidden_size = predlast_hidden_size
        self.meta_size = meta_size
        self.out_classes = out_classes
        self.inv_0 = deepsetlayers.InvLinear(hidden_size_0, hidden_size_0)
        self.inv_1 = deepsetlayers.InvLinear(2 * hidden_size_1, 4 * hidden_size_1)
        self.equiv_0 = deepsetlayers.EquivLinear(1, hidden_size_0)
        self.equiv_1 = deepsetlayers.EquivLinear(hidden_size_0, 2 * hidden_size_1)
        self.relu = torch.nn.ReLU()
        self.regressor = torch.nn.Sequential(
            torch.nn.Linear(4 * hidden_size_1 + meta_size, 2 * predlast_hidden_size),
            torch.nn.ReLU(),
            torch.nn.Linear(2 * predlast_hidden_size, predlast_hidden_size),
            torch.nn.ReLU(),
            torch.nn.Linear(predlast_hidden_size, out_classes),
        )

    def forward(self, x, y):
        x = x.unsqueeze(-1)
        # (batch_size, N, M, 1)
        N = x.shape[1]
        x = x.flatten(0, 1)
        # (batch_size * N, M, 1)
        x = self.equiv_0(x)
        # (batch_size * N, M, hidden_size_0)
        x = self.relu(x)
        x = self.inv_0(x)
        # (batch_size * N, hidden_size_0)
        x = x.reshape(-1, N, self.hidden_size_0)
        # (batch_size, N, hidden_size_0)

        x = self.equiv_1(x)
        # (batch_size, N, hidden_size_1)
        x = self.relu(x)
        x = self.inv_1(x)
        # (batch_size, hidden_size_1)

        y = y.view(-1, self.meta_size)
        # (batch_size, meta_size)        

        x = torch.hstack([x, y])
        # (batch_size, hidden_size_1 + meta_size)
        x = self.regressor(x)
        # (batch_size, out_classes)
        return x

### model v4

In [ ]:
from deepsets import deepsetlayers
import torch.nn.functional as F

class DeepSetModelV4(torch.nn.Module):
    def __init__(self, hidden_size_0=1, hidden_size_1=1, predlast_hidden_size=1, meta_size = 27, out_classes=1):
        super().__init__()
        self.hidden_size_0 = hidden_size_0
        self.hidden_size_1 = hidden_size_1
        self.predlast_hidden_size = predlast_hidden_size
        self.meta_size = meta_size
        self.out_classes = out_classes
        self.inv_0 = deepsetlayers.InvLinear(hidden_size_0, hidden_size_0)
        self.inv_1 = deepsetlayers.InvLinear(2 * hidden_size_1, 4 * hidden_size_1)
        self.equiv_0 = deepsetlayers.EquivLinear(1, hidden_size_0)
        self.equiv_1 = deepsetlayers.EquivLinear(hidden_size_0, 2 * hidden_size_1)
        self.relu = torch.nn.ReLU()
        self.regressor = torch.nn.Sequential(
            torch.nn.Linear(8 * hidden_size_1 + meta_size, 2 * predlast_hidden_size),
            torch.nn.ReLU(),
            torch.nn.Linear(2 * predlast_hidden_size, predlast_hidden_size),
            torch.nn.ReLU(),
            torch.nn.Linear(predlast_hidden_size, out_classes),
        )

    def _forward(self, x):
        x = x.unsqueeze(-1)
        # (batch_size, N, M, 1)
        N = x.shape[1]
        x = x.flatten(0, 1)
        # (batch_size * N, M, 1)
        x = self.equiv_0(x)
        # (batch_size * N, M, hidden_size_0)
        x = self.relu(x)
        x = self.inv_0(x)
        # (batch_size * N, hidden_size_0)
        x = x.reshape(-1, N, self.hidden_size_0)
        # (batch_size, N, hidden_size_0)

        x = self.equiv_1(x)
        # (batch_size, N, hidden_size_1)
        x = self.relu(x)
        x = self.inv_1(x)
        # (batch_size, hidden_size_1)
        return x
              

    def forward(self, pos, neg, y):

        pos_vec = self._forward(pos)
        neg_vec = self._forward(neg)

        y = y.view(-1, self.meta_size)
        # (batch_size, meta_size)  

        # vec0 = torch.abs(pos_vec - neg_vec)
        # vec1 = pos_vec + neg_vec
        x = torch.hstack([pos_vec, neg_vec, y])
        # x = torch.hstack([vec0, vec1, y])

        # (batch_size, hidden_size_1 + meta_size)
        x = self.regressor(x)
        # (batch_size, out_classes)
        return x

### model v5

In [6]:
from deepsets import deepsetlayers
import torch.nn.functional as F

class DeepSetModelV5(torch.nn.Module):
    def __init__(self, hidden_size_0=1, hidden_size_1=1, predlast_hidden_size=1, meta_size = 27, out_classes=1):
        super().__init__()
        self.hidden_size_0 = hidden_size_0
        self.hidden_size_1 = hidden_size_1
        self.predlast_hidden_size = predlast_hidden_size
        self.meta_size = meta_size
        self.out_classes = out_classes
        self.inv_0 = deepsetlayers.InvLinear(hidden_size_0, hidden_size_0)
        self.inv_1 = deepsetlayers.InvLinear(2 * hidden_size_1, 4 * hidden_size_1)
        self.equiv_0 = deepsetlayers.EquivLinear(1, hidden_size_0)
        self.equiv_1 = deepsetlayers.EquivLinear(hidden_size_0, 2 * hidden_size_1)
        self.relu = torch.nn.ReLU()
        self.regressor = torch.nn.Sequential(
            torch.nn.Linear(8 * hidden_size_1 + meta_size, 2 * predlast_hidden_size),
            torch.nn.BatchNorm1d(2 * predlast_hidden_size),
            torch.nn.ReLU(),
            torch.nn.Dropout(CONFIG.FIRST_DROPOUT_RATE),
            torch.nn.Linear(2 * predlast_hidden_size, predlast_hidden_size),
            torch.nn.BatchNorm1d(predlast_hidden_size),
            torch.nn.ReLU(),
            torch.nn.Dropout(CONFIG.SECOND_DROPOUT_RATE),
            torch.nn.Linear(predlast_hidden_size, out_classes),
        )

    def _forward(self, x):
        x = x.unsqueeze(-1)
        # (batch_size, N, M, 1)
        N = x.shape[1]
        x = x.flatten(0, 1)
        # (batch_size * N, M, 1)
        x = self.equiv_0(x)
        # (batch_size * N, M, hidden_size_0)
        x = self.relu(x)
        x = self.inv_0(x)
        # (batch_size * N, hidden_size_0)
        x = x.reshape(-1, N, self.hidden_size_0)
        # (batch_size, N, hidden_size_0)

        x = self.equiv_1(x)
        # (batch_size, N, hidden_size_1)
        x = self.relu(x)
        x = self.inv_1(x)
        # (batch_size, hidden_size_1)
        return x
              

    def forward(self, pos, neg, y):

        pos_vec = self._forward(pos)
        neg_vec = self._forward(neg)

        y = y.view(-1, self.meta_size)
        # (batch_size, meta_size)  

        # vec0 = torch.abs(pos_vec - neg_vec)
        # vec1 = pos_vec + neg_vec
        x = torch.hstack([pos_vec, neg_vec, y])
        # x = torch.hstack([vec0, vec1, y])

        # (batch_size, hidden_size_1 + meta_size)
        x = self.regressor(x)
        # (batch_size, out_classes)
        return x

## Data

In [ ]:
import os

import torch
from torch.utils import data
import numpy as np
from meta_gan.feature_extraction.MetaFeaturesCollector import MetaFeaturesCollector
from meta_gan.feature_extraction.LambdaFeaturesCollector import LambdaFeaturesCollector


class LambdaDatasetFolder(data.Dataset):

    def __init__(self, path: str, features_size: int, instances_size: int, classes_size: int,
                 lambdas: LambdaFeaturesCollector):
        self.root = path
        self.features = features_size
        self.instances = instances_size
        self.classes = classes_size
        paths = []
        for fname in os.listdir(self.root):
            path = os.path.join(self.root, fname)
            if not os.path.isdir(path):
                paths.append(path)
        # print(f'paths = {paths}')
        from collections import Counter
        shapes = []
        for i in paths:
            loaded_np_data = np.load(i, allow_pickle=True)
            shapes.append(loaded_np_data.shape)
            if loaded_np_data.shape == (8000, 27):
                print(i)
                exit(0)
            # print(f'loaded_np_data.shape = {loaded_np_data.}')
        print(f'shapes = {Counter(shapes)}')
        self.data_paths = paths
        self.lambda_features = lambdas

    def __getitem__(self, index) -> (torch.Tensor, torch.Tensor, torch.Tensor):
        data_path = self.data_paths[index]
        data_np = np.load(data_path, allow_pickle=True)
        dataset_tensor = torch.from_numpy(data_np).float().view((self.classes, self.instances, self.features))
        dataset_tensor = dataset_tensor.flatten(0, 1)

        lambda_tensor = self.lambda_features.get(data_np)
        return dataset_tensor, lambda_tensor

    def __len__(self):
        return len(self.data_paths)

In [ ]:
lambdaFeaturesCollector = LambdaFeaturesCollector(CONFIG.features, CONFIG.instances)

train_dataset = LambdaDatasetFolder('processed_data/processed_16_64_2/', 
                              CONFIG.features,
                              CONFIG.instances,
                              CONFIG.classes,
                              lambdaFeaturesCollector)

test_lambdaFeaturesCollector = LambdaFeaturesCollector(CONFIG.features, CONFIG.instances)

test_dataset = LambdaDatasetFolder('processed_data/test/',
                                CONFIG.features,
                                CONFIG.instances,
                                CONFIG.classes,
                                test_lambdaFeaturesCollector)

shapes = Counter({(2, 64, 16): 8000})
shapes = Counter({(2, 64, 16): 1911})


In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, 
                              batch_size=CONFIG.train_batch_size,
                              num_workers=CONFIG.num_workers,
                              shuffle=True)

test_dataloader = DataLoader(test_dataset,
                              batch_size=CONFIG.test_batch_size,
                              num_workers=CONFIG.num_workers,
                              shuffle=False)

## DataSet With Metas

In [ ]:
import torch
from torch.utils import data
import numpy as np
from meta_gan.feature_extraction.MetaFeaturesCollector import MetaFeaturesCollector
from meta_gan.feature_extraction.LambdaFeaturesCollector import LambdaFeaturesCollector
from sklearn.preprocessing import StandardScaler
import os

class DatasetWithMetaFolder(data.Dataset):

    def __init__(self, path: str, features_size: int, instances_size: int, classes_size: int,
                 meta: MetaFeaturesCollector, lambdas: LambdaFeaturesCollector, train_meta: bool,
                 meta_precalc_path: str, standardscale=False):
        self.root = path
        self.features = features_size
        self.instances = instances_size
        self.classes = classes_size
        self.meta_precalc_path = meta_precalc_path
        self.meta_precalculated = np.load(self.meta_precalc_path, allow_pickle=True)
        paths = []
        for fname in os.listdir(self.root):
            path = os.path.join(self.root, fname)
            if not os.path.isdir(path):
                paths.append(path)
        # print(f'paths = {paths}')
        from collections import Counter
        shapes = []
        for i in paths:
            loaded_np_data = np.load(i, allow_pickle=True)
            shapes.append(loaded_np_data.shape)
            if loaded_np_data.shape == (8000, 27):
                print(i)
                # exit(0)
            # print(f'loaded_np_data.shape = {loaded_np_data.}')
        print(f'shapes = {Counter(shapes)}')
        self.data_paths = paths
        self.meta_features = meta
        if train_meta:
            self.meta_features.train(self.root, load_from_fs=True)
        
        self.standardscale = standardscale
        self.lambda_features = lambdas
        self.data_scaler = StandardScaler()
        self.all_data = np.concatenate([np.load(i, allow_pickle=True) for i in self.data_paths])
        self.all_data = self.all_data.reshape((-1, self.all_data.shape[-1]))
        print(f"shape = {self.all_data.shape}")
        self.data_scaler.fit(self.all_data)
        del self.all_data

    def __getitem__(self, index) -> (torch.Tensor, torch.Tensor, torch.Tensor):
        data_path = self.data_paths[index]
        data_np = np.load(data_path, allow_pickle=True)
        dataset_tensor = torch.from_numpy(data_np).float().view((self.classes, self.instances, self.features))
        dataset_tensor = dataset_tensor.flatten(0, 1)

        # meta_tensor = self.meta_features.get(data_np)
        meta_tensor = torch.Tensor(self.meta_precalculated[index])
        meta_tensor = meta_tensor.view(self.meta_features.getLength(), 1, 1)
        lambda_tensor = self.lambda_features.get(data_np)

        if self.standardscale:
            dataset_tensor = torch.Tensor(self.data_scaler.transform(dataset_tensor))

        return dataset_tensor, meta_tensor, lambda_tensor

    def __len__(self):
        return len(self.data_paths)


In [ ]:
lambdaFeaturesCollector = LambdaFeaturesCollector(CONFIG.features, CONFIG.instances)
metaFeaturesCollector = MetaFeaturesCollector(CONFIG.features, CONFIG.instances)
train_dataset = DatasetWithMetaFolder('processed_data/processed_16_64_2/', 
                              CONFIG.features,
                              CONFIG.instances,
                              CONFIG.classes,
                              metaFeaturesCollector,
                              lambdaFeaturesCollector,
                              True,
                              'train_meta_tensors.npy',
                              standardscale=CONFIG.standardscaler)

test_lambdaFeaturesCollector = LambdaFeaturesCollector(CONFIG.features, CONFIG.instances)

test_dataset = DatasetWithMetaFolder('processed_data/test/', 
                              CONFIG.features,
                              CONFIG.instances,
                              CONFIG.classes,
                              metaFeaturesCollector,
                              lambdaFeaturesCollector,
                              False,
                              'test_meta_tensors.npy',
                              standardscale=CONFIG.standardscaler)

shapes = Counter({(2, 64, 16): 8000})
processed_data/processed_16_64_2/
Loaded from file system
shape = (1024000, 16)
shapes = Counter({(2, 64, 16): 1911})
shape = (244608, 16)


In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, 
                              batch_size=CONFIG.train_batch_size,
                              num_workers=CONFIG.num_workers,
                              shuffle=True)

test_dataloader = DataLoader(test_dataset,
                              batch_size=CONFIG.test_batch_size,
                              num_workers=CONFIG.num_workers,
                              shuffle=False)

## dataset with cached lambdas and norm



In [ ]:
import torch
from torch.utils import data
import numpy as np
from meta_gan.feature_extraction.MetaFeaturesCollector import MetaFeaturesCollector
from meta_gan.feature_extraction.LambdaFeaturesCollector import LambdaFeaturesCollector
from sklearn.preprocessing import StandardScaler
import os

class DatasetWithMetaFolder(data.Dataset):

    def __init__(self, path: str, features_size: int, instances_size: int, classes_size: int,
                 meta: MetaFeaturesCollector, lambdas: LambdaFeaturesCollector, train_meta: bool,
                 meta_precalc_path: str = None, lambda_precalc_path: str = None, 
                 standardscale=False, meta_standardscale=False, transpose=False):
        self.root = path
        self.features = features_size
        self.instances = instances_size
        self.classes = classes_size
        self.meta_precalc_path = meta_precalc_path
        self.lambda_precalc_path = lambda_precalc_path
        if self.meta_precalc_path is not None:
            self.meta_precalculated = np.load(self.meta_precalc_path, allow_pickle=True)
        if self.lambda_precalc_path is not None:
            self.lambda_precalculated = np.load(self.lambda_precalc_path, allow_pickle=True)
        self.standardscale = standardscale
        self.meta_standardscale = meta_standardscale
        self.transpose = transpose
        paths = []
        for fname in os.listdir(self.root):
            path = os.path.join(self.root, fname)
            if not os.path.isdir(path):
                paths.append(path)
        # print(f'paths = {paths}')
        from collections import Counter
        shapes = []
        for i in paths:
            loaded_np_data = np.load(i, allow_pickle=True)
            shapes.append(loaded_np_data.shape)
            if loaded_np_data.shape == (8000, 27):
                print(i)
                # exit(0)
            # print(f'loaded_np_data.shape = {loaded_np_data.}')
        print(f'shapes = {Counter(shapes)}')
        self.data_paths = paths
        self.meta_features = meta
        if train_meta:
            self.meta_features.train(self.root, load_from_fs=True)
        
        self.lambda_features = lambdas
        # standardscale
        if self.standardscale:
            self.data_scaler = StandardScaler()
            self.all_data = np.concatenate([np.load(i, allow_pickle=True) for i in self.data_paths])
            self.all_data = self.all_data.reshape((-1, self.all_data.shape[-1]))
            print(f"shape = {self.all_data.shape}")
            self.data_scaler.fit(self.all_data)
            del self.all_data

        # meta_standard_scale

        if self.meta_standardscale:
            self.meta_data_scaler = StandardScaler()
            all_meta_data = None
            if self.meta_precalc_path is not None:
                all_meta_data = self.meta_precalculated
            else:
                raise NotImplementedError(":)")
            self.meta_data_scaler.fit(all_meta_data.reshape(-1, self.meta_features.getLength()))

    def __getitem__(self, index) -> (torch.Tensor, torch.Tensor, torch.Tensor):
        data_path = self.data_paths[index]
        data_np = np.load(data_path, allow_pickle=True)
        dataset_tensor = torch.from_numpy(data_np).float().view((self.classes, self.instances, self.features))
        dataset_tensor = dataset_tensor.flatten(0, 1)

        # meta_tensor = self.meta_features.get(data_np)
        meta_tensor = torch.Tensor(self.meta_precalculated[index])
        meta_tensor = meta_tensor.view(self.meta_features.getLength(), 1, 1)
        lambda_tensor = torch.Tensor(self.lambda_precalculated[index])

        if self.standardscale:
            dataset_tensor = torch.Tensor(self.data_scaler.transform(dataset_tensor))

        if self.meta_standardscale:
            meta_tensor = torch.Tensor(self.meta_data_scaler.transform(meta_tensor.reshape(-1, self.meta_features.getLength()))).reshape(-1)

        total_elems = self.classes * self.instances
        dataset_tensor = torch.cat([dataset_tensor, 
                   (torch.arange(0, total_elems) < total_elems / 2).reshape((total_elems, 1)), 
                   (torch.arange(0, total_elems) >= total_elems / 2).reshape((total_elems, 1))], 
                  dim=1)
        if self.transpose:
            dataset_tensor = dataset_tensor.T
        return dataset_tensor, meta_tensor, lambda_tensor

    def __len__(self):
        return len(self.data_paths)

    def precalc(self, pref_name):
        from tqdm import tqdm
        meta_tensors = []
        for i in tqdm(range(self.__len__()), total=self.__len__()):
            data_path = self.data_paths[i]
            data_np = np.load(data_path, allow_pickle=True)

            meta_tensor = self.meta_features.get(data_np)
            meta_tensor = meta_tensor.view(self.meta_features.getLength(), 1, 1)
            meta_tensors.append(meta_tensor)
        
        # write meta_tensors to numpy_file
        meta_tensors = torch.stack(meta_tensors).numpy()
        np.save(f'{pref_name}_meta_tensors.npy', meta_tensors)

    def precalc_lambda(self, pref_name):
        from tqdm import tqdm
        lambda_tensors = []
        for i in tqdm(range(self.__len__()), total=self.__len__()):
            data_path = self.data_paths[i]
            data_np = np.load(data_path, allow_pickle=True)

            lambda_tensor = self.lambda_features.get(data_np)
            lambda_tensors.append(lambda_tensor)
            
        # write meta_tensors to numpy_file
        lambda_tensors = torch.stack(lambda_tensors).numpy()
        print(lambda_tensors.shape)
        np.save(f'{pref_name}_lambda_tensors.npy', lambda_tensors)


from copy import deepcopy

lambdaFeaturesCollector = LambdaFeaturesCollector(CONFIG.features, CONFIG.instances)
metaFeaturesCollector = MetaFeaturesCollector(CONFIG.features, CONFIG.instances)
train_dataset = DatasetWithMetaFolder('processed_data/processed_16_64_2/', 
                              CONFIG.features,
                              CONFIG.instances,
                              CONFIG.classes,
                              metaFeaturesCollector,
                              lambdaFeaturesCollector,
                              True,
                              meta_precalc_path='train_meta_tensors.npy',
                              lambda_precalc_path='train_lambda_lambda_tensors.npy',
                              standardscale=CONFIG.standardscaler,
                              meta_standardscale=CONFIG.meta_standardscaler,
                              transpose=CONFIG.transpose)

test_lambdaFeaturesCollector = LambdaFeaturesCollector(CONFIG.features, CONFIG.instances)

test_dataset = DatasetWithMetaFolder('processed_data/test/', 
                              CONFIG.features,
                              CONFIG.instances,
                              CONFIG.classes,
                              metaFeaturesCollector,
                              lambdaFeaturesCollector,
                              False,
                              meta_precalc_path='test_meta_tensors.npy',
                              lambda_precalc_path='test_lambda_lambda_tensors.npy',
                              standardscale=CONFIG.standardscaler,
                              meta_standardscale=CONFIG.meta_standardscaler,
                              transpose=CONFIG.transpose)

test_dataset.data_scaler = deepcopy(train_dataset.data_scaler)
test_dataset.meta_data_scaler = deepcopy(test_dataset.meta_data_scaler)

from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, 
                              batch_size=CONFIG.train_batch_size,
                              num_workers=CONFIG.num_workers,
                              shuffle=True)

test_dataloader = DataLoader(test_dataset,
                              batch_size=CONFIG.test_batch_size,
                              num_workers=CONFIG.num_workers,
                              shuffle=False)

## dataset with labels



In [7]:
import torch
from torch.utils import data
import numpy as np
from meta_gan.feature_extraction.MetaFeaturesCollector import MetaFeaturesCollector
from meta_gan.feature_extraction.LambdaFeaturesCollector import LambdaFeaturesCollector
from sklearn.preprocessing import StandardScaler
import os

class DatasetWithTargets(data.Dataset):

    def __init__(self, path: str, features_size: int, instances_size: int, classes_size: int,
                 meta: MetaFeaturesCollector, lambdas: LambdaFeaturesCollector, train_meta: bool,
                 meta_precalc_path: str = None, lambda_precalc_path: str = None, 
                 standardscale=False, meta_standardscale=False, transpose=False):
        self.root = path
        self.features = features_size
        self.instances = instances_size
        self.classes = classes_size
        self.meta_precalc_path = meta_precalc_path
        self.lambda_precalc_path = lambda_precalc_path
        if self.meta_precalc_path is not None:
            self.meta_precalculated = np.load(self.meta_precalc_path, allow_pickle=True)
        if self.lambda_precalc_path is not None:
            self.lambda_precalculated = np.load(self.lambda_precalc_path, allow_pickle=True)
        self.standardscale = standardscale
        self.meta_standardscale = meta_standardscale
        self.transpose = transpose
        paths = []
        for fname in os.listdir(self.root):
            path = os.path.join(self.root, fname)
            if not os.path.isdir(path):
                paths.append(path)
        # print(f'paths = {paths}')
        from collections import Counter
        shapes = []
        for i in paths:
            loaded_np_data = np.load(i, allow_pickle=True)
            shapes.append(loaded_np_data.shape)
            if loaded_np_data.shape == (8000, 27):
                print(i)
                # exit(0)
            # print(f'loaded_np_data.shape = {loaded_np_data.}')
        print(f'shapes = {Counter(shapes)}')
        self.data_paths = paths
        self.meta_features = meta
        if train_meta:
            self.meta_features.train(self.root, load_from_fs=True)
        
        self.lambda_features = lambdas
        # standardscale
        if self.standardscale:
            self.data_scaler = StandardScaler()
            self.all_data = np.concatenate([np.load(i, allow_pickle=True) for i in self.data_paths])
            self.all_data = self.all_data.reshape((-1, self.all_data.shape[-1]))
            print(f"shape = {self.all_data.shape}")
            self.data_scaler.fit(self.all_data)
            del self.all_data

        # meta_standard_scale

        if self.meta_standardscale:
            self.meta_data_scaler = StandardScaler()
            all_meta_data = None
            if self.meta_precalc_path is not None:
                all_meta_data = self.meta_precalculated
            else:
                raise NotImplementedError(":)")
            self.meta_data_scaler.fit(all_meta_data.reshape(-1, self.meta_features.getLength()))

    def __getitem__(self, index) -> (torch.Tensor, torch.Tensor, torch.Tensor):
        data_path = self.data_paths[index]
        data_np = np.load(data_path, allow_pickle=True)
        dataset_tensor = torch.from_numpy(data_np).float().view((self.classes, self.instances, self.features))
        dataset_tensor = dataset_tensor.flatten(0, 1)

        # meta_tensor = self.meta_features.get(data_np)
        meta_tensor = torch.Tensor(self.meta_precalculated[index])
        meta_tensor = meta_tensor.view(self.meta_features.getLength(), 1, 1)
        lambda_tensor = torch.Tensor(self.lambda_precalculated[index])

        if self.standardscale:
            dataset_tensor = torch.Tensor(self.data_scaler.transform(dataset_tensor))

        if self.meta_standardscale:
            meta_tensor = torch.Tensor(self.meta_data_scaler.transform(meta_tensor.reshape(-1, self.meta_features.getLength()))).reshape(-1)

        total_elems = self.classes * self.instances
        labels = (torch.arange(0, total_elems) < total_elems / 2).reshape(total_elems)
                   
    
        dataset_tensor = torch.cat([dataset_tensor, 
                #    (torch.arange(0, total_elems) < total_elems / 2).reshape((total_elems, 1)), 
                #    (torch.arange(0, total_elems) >= total_elems / 2).reshape((total_elems, 1))
                   ], 
                  dim=1)
        
        pos_ = dataset_tensor[labels]
        neg_ = dataset_tensor[~labels]
        
        if self.transpose:
            dataset_tensor = dataset_tensor.T
            pos_ = pos_.T
            neg_ = neg_.T
        return pos_, neg_, meta_tensor, lambda_tensor

    def __len__(self):
        return len(self.data_paths)

    def precalc(self, pref_name):
        from tqdm import tqdm
        meta_tensors = []
        for i in tqdm(range(self.__len__()), total=self.__len__()):
            data_path = self.data_paths[i]
            data_np = np.load(data_path, allow_pickle=True)

            meta_tensor = self.meta_features.get(data_np)
            meta_tensor = meta_tensor.view(self.meta_features.getLength(), 1, 1)
            meta_tensors.append(meta_tensor)
        
        # write meta_tensors to numpy_file
        meta_tensors = torch.stack(meta_tensors).numpy()
        np.save(f'{pref_name}_meta_tensors.npy', meta_tensors)

    def precalc_lambda(self, pref_name):
        from tqdm import tqdm
        lambda_tensors = []
        for i in tqdm(range(self.__len__()), total=self.__len__()):
            data_path = self.data_paths[i]
            data_np = np.load(data_path, allow_pickle=True)

            lambda_tensor = self.lambda_features.get(data_np)
            lambda_tensors.append(lambda_tensor)
            
        # write meta_tensors to numpy_file
        lambda_tensors = torch.stack(lambda_tensors).numpy()
        print(lambda_tensors.shape)
        np.save(f'{pref_name}_lambda_tensors.npy', lambda_tensors)


from copy import deepcopy

lambdaFeaturesCollector = LambdaFeaturesCollector(CONFIG.features, CONFIG.instances)
metaFeaturesCollector = MetaFeaturesCollector(CONFIG.features, CONFIG.instances)
train_dataset = DatasetWithTargets('processed_data/processed_16_64_2/', 
                              CONFIG.features,
                              CONFIG.instances,
                              CONFIG.classes,
                              metaFeaturesCollector,
                              lambdaFeaturesCollector,
                              True,
                              meta_precalc_path='train_meta_tensors.npy',
                              lambda_precalc_path='train_lambda_lambda_tensors.npy',
                              standardscale=CONFIG.standardscaler,
                              meta_standardscale=CONFIG.meta_standardscaler,
                              transpose=CONFIG.transpose)

test_lambdaFeaturesCollector = LambdaFeaturesCollector(CONFIG.features, CONFIG.instances)

test_dataset = DatasetWithTargets('processed_data/test/', 
                              CONFIG.features,
                              CONFIG.instances,
                              CONFIG.classes,
                              metaFeaturesCollector,
                              lambdaFeaturesCollector,
                              False,
                              meta_precalc_path='test_meta_tensors.npy',
                              lambda_precalc_path='test_lambda_lambda_tensors.npy',
                              standardscale=CONFIG.standardscaler,
                              meta_standardscale=CONFIG.meta_standardscaler,
                              transpose=CONFIG.transpose)

test_dataset.data_scaler = deepcopy(train_dataset.data_scaler)
test_dataset.meta_data_scaler = deepcopy(test_dataset.meta_data_scaler)

from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, 
                              batch_size=CONFIG.train_batch_size,
                              num_workers=CONFIG.num_workers,
                              shuffle=True)

test_dataloader = DataLoader(test_dataset,
                              batch_size=CONFIG.test_batch_size,
                              num_workers=CONFIG.num_workers,
                              shuffle=False)

shapes = Counter({(2, 64, 16): 8000})
processed_data/processed_16_64_2/
Loaded from file system
shape = (1024000, 16)
shapes = Counter({(2, 64, 16): 1911})
shape = (244608, 16)


## Init Model + Optimizer + Criterion

In [8]:
model = DeepSetModelV5(hidden_size_0=CONFIG.hidden_size_0, 
                     hidden_size_1=CONFIG.hidden_size_1, 
                     predlast_hidden_size=CONFIG.predlast_hidden_size, 
                     meta_size=CONFIG.meta_size,
                     out_classes=CONFIG.out_classes)

model.to(CONFIG.device)

optimizer = torch.optim.Adam(model.parameters(), lr=CONFIG.learning_rate)
# criterion = torch.nn.MSELoss()
criterion = torch.nn.BCEWithLogitsLoss()

In [9]:
 !pip install wandb==0.12.20

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 75.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 KB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 9.7 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8807 sha256=acf533f6d4edf9acf74c82b22c57ba71383fea37f3439fcc85ec5b862d5d4296
  Stored in directory: /root/.cache/pip/wheels/b7/0a/67/ada2a22079218c75a88361c0782855cc72aebc4d18d0289d05
Successfully built pathtools


In [ ]:
dict(CONFIG.__dict__)

{'__module__': '__main__',
 'features': 16,
 'instances': 64,
 'classes': 2,
 'num_epochs': 20,
 'train_batch_size': 1024,
 'learning_rate': 0.001,
 'criterion': 'torch.nn.BCEWithLogitsLoss',
 'test_batch_size': 1024,
 'hidden_size_0': 16,
 'hidden_size_1': 32,
 'predlast_hidden_size': 64,
 'meta_size': 27,
 'out_classes': 3,
 'standardscaler': True,
 'meta_standardscaler': True,
 'transpose': True,
 'device_mps_or_cpu': 'cpu',
 'device': device(type='cuda', index=0),
 'seed': 42,
 'num_workers': 8,
 '__dict__': <attribute '__dict__' of 'CONFIG' objects>,
 '__weakref__': <attribute '__weakref__' of 'CONFIG' objects>,
 '__doc__': None}

In [20]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

count_parameters(model)

841219

In [10]:
# init wandb

import wandb

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

cfg = dict(CONFIG.__dict__)
cfg.pop('__module__')
cfg.pop('__dict__')
cfg.pop('__weakref__')
cfg.pop('__doc__')
cfg['parameteres_num'] = count_parameters(model)

def get_accuracy(threshold):
    def lambda_accuracy(y_pred, y_true):
        res = ((y_pred >= threshold) & (y_true.to(bool))).sum(dim=1).clamp(min=0, max=1)
        return res.sum() / res.shape[0]
    return lambda_accuracy

accs_dict = { f'acc_{int(100 * round(i, 1))}': get_accuracy(i) for i in np.linspace(0, 1, 11)}
accs_dict['acc_101'] = get_accuracy(1.01)

print(cfg)

run = wandb.init(project="diploma1234", config=cfg, id='50__v5_fixed_acc_16_without_target', notes='modelv5, concat simple pos and neg vectors, lr x2, meta_and_ds_ftrs_normx2_bce_v5_trnsps_2xlr_shared_scaler_h4x_50_epochs, fixed accuracy, dropout and batchnorm, 16 sample features dim, without 2 target columns')

run.watch(model)

{'features': 16, 'instances': 64, 'classes': 2, 'num_epochs': 50, 'train_batch_size': 1024, 'learning_rate': 0.002, 'criterion': 'torch.nn.BCEWithLogitsLoss', 'FIRST_DROPOUT_RATE': 0.7, 'SECOND_DROPOUT_RATE': 0.5, 'test_batch_size': 1024, 'hidden_size_0': 64, 'hidden_size_1': 128, 'predlast_hidden_size': 256, 'meta_size': 27, 'out_classes': 3, 'standardscaler': True, 'meta_standardscaler': True, 'transpose': True, 'device_mps_or_cpu': 'cpu', 'device': device(type='cuda', index=0), 'seed': 42, 'num_workers': 8, 'parameteres_num': 841219}


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [11]:
def acc(y_pred, y_true):
    y_pred_ = (y_pred == y_pred.max(dim=1, keepdim=True)[0]).to(bool)
    y_pred__ = ((y_true == 1) & (y_pred_ == 1)).to(int)
    # print(y_pred__)
    return y_pred__.sum() / y_pred__.shape[0]

## train

In [12]:
from tqdm.notebook import tqdm

In [26]:
!mkdir checkpoints

In [13]:
for epoch in range(CONFIG.num_epochs):
    # train loop
    train_epoch_loss = 0
    train_accs = {k: 0 for (k, v) in accs_dict.items()}
    train_correct_acc = 0
    model.train()
    for (ind, (X_pos, X_neg, meta, y)) in tqdm(enumerate(train_dataloader), total=len(train_dataloader), desc=f"Train Epoch {epoch}"):
        X_pos = X_pos.to(CONFIG.device)
        X_neg = X_neg.to(CONFIG.device)
        meta = meta.to(CONFIG.device)
        y = y.to(CONFIG.device)

        optimizer.zero_grad()
        y_pred = model(X_pos, X_neg, meta)
        loss = criterion(y_pred, y)
        loss.backward()
        optimizer.step()

        train_epoch_loss += loss.item()
        for (k, v) in accs_dict.items():
            train_accs[k] += v(torch.sigmoid(y_pred), y)
        train_correct_acc += acc(torch.sigmoid(y_pred), y) * y.shape[0]
    train_correct_acc /= len(train_dataset)
    train_epoch_loss = train_epoch_loss / len(train_dataloader)
    train_accs = {f'train_{k}': v / len(train_dataloader) for (k, v) in train_accs.items()}

    # test loop
    model.eval()
    test_epoch_loss = 0
    test_squared_error = 0
    test_total_elems = 0
    test_accs = {k: 0 for (k, v) in accs_dict.items()}
    test_correct_acc = 0
    for (ind, (X_pos, X_neg, meta, y)) in tqdm(enumerate(test_dataloader), total=len(test_dataloader), desc=f"Test Epoch {epoch}"):
        X_pos = X_pos.to(CONFIG.device)
        X_neg = X_neg.to(CONFIG.device)
        meta = meta.to(CONFIG.device)
        y = y.to(CONFIG.device)

        y_pred = model(X_pos, X_neg, meta)
        loss = criterion(y_pred, y)

        test_epoch_loss += loss.item()

        y_pred_ = torch.sigmoid(y_pred)

        # update test squared error without mean
        test_squared_error += torch.sum((y_pred_ - y) ** 2).item()
        test_total_elems += y_pred.shape[0] * y_pred.shape[1]

        for (k, v) in accs_dict.items():
            test_accs[k] += v(torch.sigmoid(y_pred), y)
        test_correct_acc += acc(torch.sigmoid(y_pred), y) * y.shape[0]
    test_accs = {f'test_{k}': v / len(test_dataloader) for (k, v) in test_accs.items()}
    test_correct_acc /= len(test_dataset)



    test_epoch_loss = test_epoch_loss / len(test_dataloader)
    test_mse = test_squared_error / test_total_elems
    log_dict = dict({"train_loss": train_epoch_loss,
               "test_loss": test_epoch_loss,
               "test_mse": test_mse}, **train_accs)
    log_dict = dict(log_dict, **test_accs)

    log_dict['train_correct_acc'] = train_correct_acc
    log_dict['test_correct_acc'] = test_correct_acc

    wandb.log(log_dict)
    
    torch.save(model.state_dict(), f'discriminator-deepset-16_64_2-{epoch + 1}.pkl')


    print(f'Epoch {epoch} train loss: {train_epoch_loss}')
    print(f'Epoch {epoch} test loss: {test_epoch_loss}')
    print(f'Epoch {epoch} test mse: {test_mse}')
    print()

wandb.finish()

Train Epoch 0:   0%|          | 0/8 [00:00<?, ?it/s]

Test Epoch 0:   0%|          | 0/2 [00:00<?, ?it/s]

Epoch 0 train loss: 0.6432418376207352
Epoch 0 test loss: 0.6641592085361481
Epoch 0 test mse: 0.23560590226675607



Train Epoch 1:   0%|          | 0/8 [00:00<?, ?it/s]

Test Epoch 1:   0%|          | 0/2 [00:00<?, ?it/s]

...

Epoch 48 train loss: 0.5411438196897507
Epoch 48 test loss: 0.5888043642044067
Epoch 48 test mse: 0.20298165840855573



Train Epoch 49:   0%|          | 0/8 [00:00<?, ?it/s]

Test Epoch 49:   0%|          | 0/2 [00:00<?, ?it/s]

Epoch 49 train loss: 0.543506808578968
Epoch 49 test loss: 0.5896568298339844
Epoch 49 test mse: 0.20345947435965572



test_acc_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_acc_10,███▇▇▆▆▅▅▆▅▆▄▅▆▆▃▄▅▆▄▄▄▄▄▄▄▅▂▄▁▄▅▅▄▄▄▃▃▁
test_acc_100,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_acc_101,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_acc_20,███▇▇▅▆▄▅▅▅▅▅▅▄▃▄▄▄▃▄▄▅▅▂▃▂▃▂▃▁▂▃▃▂▂▃▂▂▁
test_acc_30,█▄▃▃▃▃▄▂▃▃▃▃▃▃▂▂▂▂▂▂▂▃▃▃▂▂▂▂▁▂▁▂▂▁▁▂▂▁▂▁
test_acc_40,█▁▂▂▄▄▆▅▅▆▆▅▅▆▆▅▆▆▆▆▆▆▆▆▅▆▅▆▅▆▅▆▆▅▆▅▆▅▆▅
test_acc_50,▃▁▂▄▆▇▇▇▇▇▇█▇█▇██████████████▇██████████
test_acc_60,▁▁▂▄▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇█▇▇▇▇▇██▇▇▇▇▇▇▇█
test_acc_70,▁▁▁▂▄▅▅▆▆▆▆▇▆▆▆▇▇▇▇▇▆▇▇▇█▇▇█▇▇██▇▇▇▇▇▇▇█
test_acc_80,▁▁▁▁▂▃▃▅▅▅▄▅▄▅▄▅▆▆▅▅▆▆▆▆▇▆▇▇▇▇▇█▇▇▇▇▇▇█▇


In [16]:
!mkdir checkpoints

In [17]:
!mv discriminator* checkpoints/

In [18]:
!zip checkpoints-16.zip -r checkpoints/ 

  adding: checkpoints/ (stored 0%)
  adding: checkpoints/discriminator-deepset-16_64_2-38.pkl (deflated 8%)
  adding: checkpoints/discriminator-deepset-16_64_2-31.pkl (deflated 8%)
  adding: checkpoints/discriminator-deepset-16_64_2-12.pkl (deflated 8%)
  adding: checkpoints/discriminator-deepset-16_64_2-14.pkl (deflated 8%)
  adding: checkpoints/discriminator-deepset-16_64_2-18.pkl (deflated 8%)
  adding: checkpoints/discriminator-deepset-16_64_2-24.pkl (deflated 8%)
  adding: checkpoints/discriminator-deepset-16_64_2-16.pkl (deflated 8%)
  adding: checkpoints/discriminator-deepset-16_64_2-27.pkl (deflated 8%)
  adding: checkpoints/discriminator-deepset-16_64_2-8.pkl (deflated 8%)
  adding: checkpoints/discriminator-deepset-16_64_2-15.pkl (deflated 8%)
  adding: checkpoints/discriminator-deepset-16_64_2-50.pkl (deflated 7%)
  adding: checkpoints/discriminator-deepset-16_64_2-46.pkl (deflated 7%)
  adding: checkpoints/discriminator-deepset-16_64_2-43.pkl (deflated 8%)
  adding: checkpo